# Grid Serialization Examples

Persist `Grid` instances as JSON, for example when you need reproducible fixtures or want to share datasets across services.
These snippets focus on the developer-facing `Grid.serialize()` / `Grid.deserialize()` workflow.


## Build a Minimal Grid

Start with a compact network so each serialization step stays readable.

In [ ]:
from power_grid_model_ds import Grid
from power_grid_model_ds.arrays import LineArray, NodeArray, SourceArray, SymLoadArray
from power_grid_model_ds.enums import NodeType

grid = Grid.empty()

substation = NodeArray(id=[1001], u_rated=[10_500.0], node_type=[NodeType.SUBSTATION_NODE.value])
grid.append(substation, check_max_id=False)

feeders = NodeArray(id=[1002, 1003], u_rated=[10_500.0] * 2, node_type=[NodeType.UNSPECIFIED.value] * 2)
grid.append(feeders, check_max_id=False)

lines = LineArray(
    id=[2001, 2002],
    from_node=[1001, 1002],
    to_node=[1002, 1003],
    from_status=[1, 1],
    to_status=[1, 1],
    i_n=[200.0, 200.0],
    r1=[0.08, 0.12],
    x1=[0.03, 0.04],
    c1=[0.0, 0.0],
    tan1=[0.0, 0.0],
)
grid.append(lines, check_max_id=False)

load = SymLoadArray(
    id=[3001],
    node=[1003],
    type=[1],
    p_specified=[750_000.0],
    q_specified=[325_000.0],
    status=[1],
)
grid.append(load, check_max_id=False)

source = SourceArray(id=[4001], node=[1001], status=[1], u_ref=[0.0])
grid.append(source, check_max_id=False)

grid.check_ids()
grid

## Serialize to JSON

Use `Grid.serialize()` to dump the container. All JSON settings (indentation, ordering, ASCII handling, etc.) can be forwarded to `json.dump`.
Set `strict=False` if you add custom, non-serializable attributes and want to skip them instead of raising.


In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

workspace = TemporaryDirectory()
json_path = Path(workspace.name) / "grid_snapshot.json"

grid.serialize(json_path, indent=2)

print(json_path.read_text())

## Deserialize and Validate

Load the JSON as a Grid with `Grid.deserialize()` and compare it with the in-memory object to verify that every array survived the round trip.


In [ ]:
reloaded_grid = Grid.deserialize(json_path)

print(f"Arrays identical: {reloaded_grid == grid}")
print(f"Nodes restored: {reloaded_grid.node.size}")
print(f"Serialized file: {json_path}")

workspace.cleanup()

## PGM Compatibility

Use `PowerGridModelInterface` when you need JSON that the core `power-grid-model` solver accepts directly. The sequence is simple: create the solver payload from your grid, dump it with `json_serialize_to_file()`, reload it back into a `Grid`.

In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

from power_grid_model.utils import json_serialize_to_file

from power_grid_model_ds import PowerGridModelInterface

compat_workspace = TemporaryDirectory()
pgm_path = Path(compat_workspace.name) / "pgm_input.json"

pgm_payload = PowerGridModelInterface(grid).create_input_from_grid()

# Note: replace NaN values before serializing, otherwise json_serialize_to_file will drop those columns entirely.
json_serialize_to_file(pgm_path, pgm_payload)

restored_grid = Grid.deserialize(pgm_path)
restored_payload = PowerGridModelInterface(restored_grid).create_input_from_grid()

compat_workspace.cleanup()